In [1]:
import pandas as pd
import numpy as np
from pandasql import sqldf
from collections import defaultdict
import requests
from bs4 import BeautifulSoup
from flask import Flask, jsonify, request
from flask_cors import CORS
from functools import cache

In [2]:
url = 'https://www.jpmorganchase.com/about/our-business/historical-prime-rate'

# Fetch the HTML content of the webpage
response = requests.get(url)
html_content = response.content

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')

# Use BeautifulSoup to find the relevant HTML elements containing the interest rates
# Replace 'tag', 'class', and 'id' with the actual HTML tags, classes, or IDs from the webpage
interest_rates = soup.find_all('td')

# Extract and print the interest rates
for rate in interest_rates:
    print(rate.text)

7/27/2023
8.50%
5/4/2023
8.25%
3/23/2023
8.00%
2/2/2023
7.75%
12/15/2022
7.50%
11/3/2022
7.00%
9/22/2022

6.25%
7/28/2022
5.50%
6/16/2022
4.75%
5/5/2022
4.00%
3/17/2022
3.50%
3/16/2020
3.25%
3/4/2020
4.25%
10/31/2019
4.75%
9/19/2019
5.00%
8/1/2019
5.25%
12/20/2018
5.5%
9/27/2018
5.25%
6/14/2018
5.00%
3/22/2018
4.75%
12/14/2017
4.50%
6/15/2017
4.25%
3/16/2017
4.00%
12/15/2016
3.75%
12/17/2015
3.50%
12/16/2008
3.25%
10/29/2008
4.00%
10/8/2008
4.50%
4/30/2008
5.00%
3/18/2008
5.25%
1/30/2008
6.00%
1/22/2008
6.50%
12/11/2007
7.25%
10/31/2007
7.50%
9/18/2007
7.75%
6/29/2006
8.25%
5/10/2006
8.00%
3/28/2006
7.75%
1/31/2006
7.50%
12/13/2005
7.25%
11/1/2005
7.00%
9/20/2005
6.75%
8/9/2005
6.50%
6/30/2005
6.25%
5/3/2005
6.00%
3/22/2005
5.75%
2/2/2005
5.50%
12/14/2004
5.25%
11/10/2004
5.00%
9/21/2004
4.75%
8/10/2004
4.50%
6/30/2004
4.25%
6/27/2003
4.00%
11/7/2002
4.25%
12/12/2001
4.75%
11/7/2001
5.00%
10/3/2001
5.50%
9/17/2001
6.00%
8/22/2001
6.50%
6/28/2001
6.75%
5/16/2001
7.00%
4/19/2001
7.50%
3/

In [3]:
df = pd.read_excel("data.xlsx")
#df

In [4]:
columns = ['Date', 'Ticker', 'Company', 'Net_Income', 'Revenue', 'Earnings_Date']
df.columns = columns
#df

In [5]:
daily = pd.read_csv('daily.csv')
daily = daily.iloc[:,:5]
daily

,PERMNO,date,TICKER,COMNAM,PRC
0,10104,2000-01-03,ORCL,ORACLE CORP,118.12500
1,10104,2000-01-04,ORCL,ORACLE CORP,107.68750
2,10104,2000-01-05,ORCL,ORACLE CORP,102.00000
3,10104,2000-01-06,ORCL,ORACLE CORP,96.00000
4,10104,2000-01-07,ORCL,ORACLE CORP,103.37500
...,...,...,...,...,...
2949042,93436,2023-12-22,TSLA,TESLA INC,252.53999
2949043,93436,2023-12-26,TSLA,TESLA INC,256.60999
2949044,93436,2023-12-27,TSLA,TESLA INC,261.44000
2949045,93436,2023-12-28,TSLA,TESLA INC,253.17999


In [6]:
daily

,PERMNO,date,TICKER,COMNAM,PRC
0,10104,2000-01-03,ORCL,ORACLE CORP,118.12500
1,10104,2000-01-04,ORCL,ORACLE CORP,107.68750
2,10104,2000-01-05,ORCL,ORACLE CORP,102.00000
3,10104,2000-01-06,ORCL,ORACLE CORP,96.00000
4,10104,2000-01-07,ORCL,ORACLE CORP,103.37500
...,...,...,...,...,...
2949042,93436,2023-12-22,TSLA,TESLA INC,252.53999
2949043,93436,2023-12-26,TSLA,TESLA INC,256.60999
2949044,93436,2023-12-27,TSLA,TESLA INC,261.44000
2949045,93436,2023-12-28,TSLA,TESLA INC,253.17999


In [7]:
df

,Date,Ticker,Company,Net_Income,Revenue,Earnings_Date
0,2000-03-31,AAL,AMERICAN AIRLINES GROUP INC,132.0,4577.0,2000-04-19
1,2000-06-30,AAL,AMERICAN AIRLINES GROUP INC,321.0,5011.0,2000-07-19
2,2000-09-30,AAL,AMERICAN AIRLINES GROUP INC,313.0,5256.0,2000-10-18
3,2000-12-31,AAL,AMERICAN AIRLINES GROUP INC,47.0,4859.0,2001-01-17
4,2001-03-31,AAL,AMERICAN AIRLINES GROUP INC,-43.0,4760.0,2001-04-18
...,...,...,...,...,...,...
44587,2022-12-31,ALLE,ALLEGION PLC,135.3,861.5,2023-02-22
44588,2023-03-31,ALLE,ALLEGION PLC,123.5,923.0,2023-04-26
44589,2023-06-30,ALLE,ALLEGION PLC,142.0,912.5,2023-07-26
44590,2023-09-30,ALLE,ALLEGION PLC,156.3,917.9,2023-10-31


In [8]:
def getUrl(function, symbol):
    apikeyPath = 'apikey.txt'
    file = open(apikeyPath, 'r')
    apikey = file.readline().strip()
    file.close()

    url = f"https://www.alphavantage.co/query?function={function}&symbol={symbol}&apikey={apikey}" if function != 'TIME_SERIES_DAILY' else f"https://www.alphavantage.co/query?function={function}&symbol={symbol}&outputsize=full&apikey={apikey}"
    return url

In [9]:
@cache
def getDataFromAlphaVantage(url):
    return requests.get(url)

In [10]:
def createDailyDf(data, TICKER):
    result = pd.DataFrame(data['Time Series (Daily)']).T
    result.reset_index(inplace=True)
    result.rename(columns={'index': 'date', '4. close': 'PRC'}, inplace=True)
    result['TICKER'] = TICKER
    result['date'] = pd.to_datetime(result['date'])
    result['PRC'] = result['PRC'].astype(float)
    result = result.sort_values(by='date')
    return result

In [11]:
def createStatsDf(earningsDf, incomeDf, TICKER): 
    df = pd.merge(earningsDf, incomeDf, on='fiscalDateEnding')
    df['Ticker'] = TICKER
    df.rename(columns={'fiscalDateEnding': 'Date', 'reportedDate': 'Earnings_Date', 'netIncome': 'Net_Income', 'totalRevenue': 'Revenue'}, inplace=True)
    df['Date'] = pd.to_datetime(df['Date'])     
    df['Earnings_Date'] = pd.to_datetime(df['Earnings_Date'])
    df['Net_Income'] = df['Net_Income'].astype(np.int64)
    df['Revenue'] = df['Revenue'].astype(np.int64)
    df = df.sort_values(by='Earnings_Date')
    #Net_Income, Revenue, Earnings_Date,Date
    return df

In [12]:
def createFinalDf(df, daily):
    df['Quarter_End'] = df['Earnings_Date'].shift(-1) - pd.Timedelta(days=1)
    df.loc[df['Ticker'] != df['Ticker'].shift(1), 'Quarter_End'] = pd.NA

    df['Date'] = pd.to_datetime(df['Date'])
    daily['date'] = pd.to_datetime(daily['date'])
    df = pd.merge(df, daily[['date', 'TICKER', 'PRC']], left_on=['Earnings_Date', 'Ticker'], right_on=['date', 'TICKER'])

    df = df.rename(columns={'PRC': 'prc1'})
    df = df.drop(columns=['date', 'TICKER'])

    df = pd.merge(df, daily[['date', 'TICKER', 'PRC']], left_on=['Quarter_End', 'Ticker'], right_on=['date', 'TICKER'])
    df = df.rename(columns={'PRC': 'prc2'})
    df = df.drop(columns=['date', 'TICKER'])

    df['ret'] = (df['prc2']/df['prc1']) - 1

    df['Earn_Increase'] = (df['Net_Income'] - df['Net_Income'].shift(4)) / abs(df['Net_Income'].shift(4))
    df['Rev_Increase'] = (df['Revenue'] / df['Revenue'].shift(4)) - 1
    df.loc[df['Ticker'] != df['Ticker'].shift(1), 'Earn_Increase'] = pd.NA
    df.loc[df['Ticker'] != df['Ticker'].shift(1), 'Rev_Increase'] = pd.NA

    df = df.dropna(subset=['Earn_Increase', 'Rev_Increase'])
    df.reset_index(drop=True, inplace=True)

    increases = np.zeros(len(df))

    # Loop through the DataFrame to count increases
    for i in range(1, len(increases)):
        #print(i)
        if df.loc[i, 'Earn_Increase'] > df.loc[i-1, 'Earn_Increase'] and df.loc[i, 'Ticker'] == df.loc[i-1, 'Ticker']:
            increases[i] = increases[i-1] + 1
        else:
            increases[i] = 0
    df['Earn_Increase_Count'] = increases

    increases = np.zeros(len(df))

    # Loop through the DataFrame to count increases
    for i in range(1, len(increases)):
        #print(i)
        if df.loc[i, 'Rev_Increase'] > df.loc[i-1, 'Rev_Increase'] and df.loc[i, 'Ticker'] == df.loc[i-1, 'Ticker']:
            increases[i] = increases[i-1] + 1
        else:
            increases[i] = 0
    df['Rev_Increase_Count'] = increases
    return df

In [13]:
df = createFinalDf(df, daily)
df

,Date,Ticker,Company,Net_Income,Revenue,Earnings_Date,Quarter_End,prc1,prc2,ret,Earn_Increase,Rev_Increase,Earn_Increase_Count,Rev_Increase_Count
0,2014-12-31,AAL,AMERICAN AIRLINES GROUP INC,597.0,10160.0,2015-01-27,2015-04-23,52.695,51.45,-0.023627,1.298351,0.384762,0.0,0.0
1,2015-03-31,AAL,AMERICAN AIRLINES GROUP INC,932.0,9827.0,2015-04-24,2015-07-23,52.705,42.61,-0.191538,0.941667,-0.016808,0.0,0.0
2,2015-06-30,AAL,AMERICAN AIRLINES GROUP INC,1704.0,10827.0,2015-07-24,2015-10-22,39.630,45.99,0.160484,0.974508,-0.046583,1.0,0.0
3,2015-09-30,AAL,AMERICAN AIRLINES GROUP INC,1693.0,10706.0,2015-10-23,2016-01-28,45.670,38.14,-0.164878,0.79724,-0.038872,0.0,1.0
4,2015-12-31,AAL,AMERICAN AIRLINES GROUP INC,3281.0,9630.0,2016-01-29,2016-04-21,38.990,40.01,0.026161,4.495812,-0.052165,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33824,2022-06-30,ALLE,ALLEGION PLC,115.1,773.1,2022-07-28,2022-10-26,104.310,97.33,-0.066916,-0.030329,0.035078,1.0,0.0
33825,2022-09-30,ALLE,ALLEGION PLC,114.6,913.7,2022-10-27,2023-02-21,103.460,115.01,0.111637,-0.201394,0.274338,0.0,1.0
33826,2022-12-31,ALLE,ALLEGION PLC,135.3,861.5,2023-02-22,2023-04-25,113.140,101.95,-0.098904,0.199468,0.214749,1.0,0.0
33827,2023-03-31,ALLE,ALLEGION PLC,123.5,923.0,2023-04-26,2023-07-25,107.490,127.07,0.182156,0.327957,0.275567,2.0,1.0


In [14]:
test = sqldf("select * from df where Ticker = 'AMD'")
test

,Date,Ticker,Company,Net_Income,Revenue,Earnings_Date,Quarter_End,prc1,prc2,ret,Earn_Increase,Rev_Increase,Earn_Increase_Count,Rev_Increase_Count
0,2000-09-30 00:00:00.000000,AMD,ADVANCED MICRO DEVICES,408.567,1206.549,2000-10-11 00:00:00.000000,2001-01-16 00:00:00.000000,21.75,17.125,-0.212644,-0.604485,-0.880433,0.0,0.0
1,2000-12-31 00:00:00.000000,AMD,ADVANCED MICRO DEVICES,177.968,1175.172,2001-01-17 00:00:00.000000,2001-04-17 00:00:00.000000,18.50,23.000,0.243243,-0.864971,-0.879432,0.0,1.0
2,2001-03-31 00:00:00.000000,AMD,ADVANCED MICRO DEVICES,124.837,1188.747,2001-04-18 00:00:00.000000,2001-07-11 00:00:00.000000,27.85,21.120,-0.241652,-0.909209,-0.880863,0.0,0.0
3,2001-06-30 00:00:00.000000,AMD,ADVANCED MICRO DEVICES,17.352,985.264,2001-07-12 00:00:00.000000,2001-10-16 00:00:00.000000,22.70,10.610,-0.532599,-0.916231,-0.158208,0.0,1.0
4,2001-09-30 00:00:00.000000,AMD,ADVANCED MICRO DEVICES,-186.928,765.870,2001-10-17 00:00:00.000000,2002-01-15 00:00:00.000000,9.71,19.150,0.972194,-1.457521,-0.365239,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,2022-06-30 00:00:00.000000,AMD,ADVANCED MICRO DEVICES,447.000,6550.000,2022-08-02 00:00:00.000000,2022-10-31 00:00:00.000000,99.29,60.060,-0.395105,-0.370423,0.701299,0.0,0.0
81,2022-09-30 00:00:00.000000,AMD,ADVANCED MICRO DEVICES,66.000,5565.000,2022-11-01 00:00:00.000000,2023-01-30 00:00:00.000000,59.66,72.450,0.214381,-0.928494,0.290285,0.0,0.0
82,2022-12-31 00:00:00.000000,AMD,ADVANCED MICRO DEVICES,21.000,5599.000,2023-01-31 00:00:00.000000,2023-05-01 00:00:00.000000,75.15,89.690,0.193480,-0.978439,0.160174,0.0,0.0
83,2023-03-31 00:00:00.000000,AMD,ADVANCED MICRO DEVICES,-139.000,5353.000,2023-05-02 00:00:00.000000,2023-07-31 00:00:00.000000,89.91,114.400,0.272383,-1.176845,-0.090708,0.0,0.0


In [15]:
# variables
earn = 0.3
rev = 0.25
increase = 1

In [16]:
df = df.sort_values(by='Date')
df

,Date,Ticker,Company,Net_Income,Revenue,Earnings_Date,Quarter_End,prc1,prc2,ret,Earn_Increase,Rev_Increase,Earn_Increase_Count,Rev_Increase_Count
14481,2000-04-30,SJM,SMUCKER (JM) CO,0.968,156.598,2000-06-15,2000-08-16,17.68750,18.00000,0.017668,1.00806,-0.832712,0.0,0.0
14482,2000-04-30,SJM,SMUCKER (JM) CO,0.968,156.598,2000-06-15,2000-08-16,17.00000,18.18750,0.069853,-0.992372,-0.837957,0.0,0.0
14483,2000-04-30,SJM,SMUCKER (JM) CO,0.968,156.598,2000-06-15,2000-08-16,17.00000,18.00000,0.058824,-0.993834,-0.837013,0.0,1.0
2138,2000-06-30,BIO,BIO-RAD LABORATORIES INC,5.074,186.826,2000-08-03,2000-11-01,23.00000,-30.00000,-2.304348,-0.979205,-0.980266,1.0,1.0
18678,2000-06-30,FCX,FREEPORT-MCMORAN INC,-9.176,397.348,2000-07-18,2000-10-16,8.50000,8.06250,-0.051471,-1.018389,-0.919484,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6915,2023-08-31,FDX,FEDEX CORP,1078.000,21681.000,2023-09-20,2023-12-18,250.52000,281.92001,0.125339,0.9319,-0.111216,2.0,0.0
26290,2023-08-31,KMX,CARMAX INC,118.635,7490.823,2023-09-28,2023-12-20,69.01000,74.68000,0.082162,-0.057742,-0.118931,3.0,2.0
23212,2023-08-31,COST,COSTCO WHOLESALE CORP,2160.000,78939.000,2023-09-26,2023-12-13,552.96002,642.00000,0.161024,0.156317,0.094991,1.0,1.0
9905,2023-08-31,LEN,LENNAR CORP,1108.996,8729.603,2023-09-14,2023-12-13,105.70000,145.16000,0.373321,0.272231,0.085076,0.0,0.0


In [17]:
df['quarter'] = pd.DatetimeIndex(df['Date']).year * 10 + (pd.DatetimeIndex(df['Date']).month // 3) + 1
df

,Date,Ticker,Company,Net_Income,Revenue,Earnings_Date,Quarter_End,prc1,prc2,ret,Earn_Increase,Rev_Increase,Earn_Increase_Count,Rev_Increase_Count,quarter
14481,2000-04-30,SJM,SMUCKER (JM) CO,0.968,156.598,2000-06-15,2000-08-16,17.68750,18.00000,0.017668,1.00806,-0.832712,0.0,0.0,20002
14482,2000-04-30,SJM,SMUCKER (JM) CO,0.968,156.598,2000-06-15,2000-08-16,17.00000,18.18750,0.069853,-0.992372,-0.837957,0.0,0.0,20002
14483,2000-04-30,SJM,SMUCKER (JM) CO,0.968,156.598,2000-06-15,2000-08-16,17.00000,18.00000,0.058824,-0.993834,-0.837013,0.0,1.0,20002
2138,2000-06-30,BIO,BIO-RAD LABORATORIES INC,5.074,186.826,2000-08-03,2000-11-01,23.00000,-30.00000,-2.304348,-0.979205,-0.980266,1.0,1.0,20003
18678,2000-06-30,FCX,FREEPORT-MCMORAN INC,-9.176,397.348,2000-07-18,2000-10-16,8.50000,8.06250,-0.051471,-1.018389,-0.919484,0.0,0.0,20003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6915,2023-08-31,FDX,FEDEX CORP,1078.000,21681.000,2023-09-20,2023-12-18,250.52000,281.92001,0.125339,0.9319,-0.111216,2.0,0.0,20233
26290,2023-08-31,KMX,CARMAX INC,118.635,7490.823,2023-09-28,2023-12-20,69.01000,74.68000,0.082162,-0.057742,-0.118931,3.0,2.0,20233
23212,2023-08-31,COST,COSTCO WHOLESALE CORP,2160.000,78939.000,2023-09-26,2023-12-13,552.96002,642.00000,0.161024,0.156317,0.094991,1.0,1.0,20233
9905,2023-08-31,LEN,LENNAR CORP,1108.996,8729.603,2023-09-14,2023-12-13,105.70000,145.16000,0.373321,0.272231,0.085076,0.0,0.0,20233


In [18]:
def generate(earnLow, earnHigh, revLow, revHigh, increase, df):
    quarters = set(df['quarter'])
    returns = defaultdict(list)
    returns.update({key: [] for key in quarters})

    stocks = defaultdict(list)
    stocks.update({key: [] for key in quarters})
    
    groups = df.groupby('quarter')
    count = 0
    for name, group in groups:
    #print(name)
        for index, row in group.iterrows():
            # row['Rev_Increase_Count'] >= 1
            # row['Rev_Increase'] >= rev
            # row['Net_Income'] > 0 
            if (row['Earn_Increase'] >= earnLow and row['Earn_Increase'] <= earnHigh) and (row['Rev_Increase'] >= revLow and row['Rev_Increase'] <= revHigh) and row['Earn_Increase_Count'] == increase:
                returns[name].append(max(row['ret'], -0.1))
                stocks[name].append(row['Ticker'])
                count += 1
    return returns, stocks, count

In [19]:
def test(returns):
    test = []
    for key, val in returns.items():
        #print(key)
        #print(np.mean(val))
        if val:
            test.append(np.nanmean(val))    
    #print(np.nanmean(test))
    return np.nanmean(test)

In [20]:
#returns, stocks = generate(0.1, 0.18, 3, df)


In [21]:
# result = test(returns)
# print(result)
# print(stocks[20212])
# print(returns[20212])
# print(np.mean(returns[20212]))

In [22]:
# maxE = 0
# maxR = 0
# maxI = 0
# maxRet = 0
# # i = earnings, j = revenue
# earnLow = 0.1
# earnHigh = float('inf')
# revLow = 0.1
# revHigh = 0.25
# i = 0.1
# j = 0.1
# # while i <= 0.6:
# resultArr = []
# rev = revLow
# while rev <= revHigh:
#     earnIncrease = 1
# #         for k in range(0, 4):
#     returns, stocks = generate(earnLow, earnHigh, rev, revHigh, earnIncrease, df)
#     result = test(returns)
#     resultArr.append((result, rev))
#     if result and result > maxRet:
#         maxRet = result
#         maxE, maxR, maxI = earnLow, rev, earnIncrease
#     rev += 0.01
# i += 0.01
# print(maxE, maxR, maxI)
# print(sorted(resultArr, reverse=True))

In [23]:
def rangeSearch(earnLow, earnHigh, revLow, revHigh, earnIncrease):
    maxRet = 0

    returns, stocks, count = generate(earnLow, earnHigh, revLow, revHigh, earnIncrease, df)
    result = test(returns)
    if result and result > maxRet:
        maxRet = result
    return result, count

In [24]:
returns, sampleSize = rangeSearch(0.2, 0.4, 0.17, 0.19, 2)
print("returns: ", returns)
print("sample size: ", sampleSize)

returns:  0.12418686993522145
sample size:  20


In [38]:
print(rangeSearch(0.23, 0.33, 0.12, 0.14, 0))

(0.03384838084633637, 64)


In [40]:
def predictStockReturns(TICKER):
    TIME_SERIES_DAILY = 'TIME_SERIES_DAILY'
    INCOME_STATEMENT = 'INCOME_STATEMENT'
    EARNINGS = 'EARNINGS'

    timeSeriesUrl = getUrl(TIME_SERIES_DAILY, TICKER)
    timeData = getDataFromAlphaVantage(timeSeriesUrl).json();

    earningsUrl = getUrl(EARNINGS, TICKER)
    earningsData = getDataFromAlphaVantage(earningsUrl).json();

    incomeUrl = getUrl(INCOME_STATEMENT, TICKER)
    incomeData = getDataFromAlphaVantage(incomeUrl).json();
    
    dailyStock = createDailyDf(timeData, TICKER)
    earningsDf = pd.DataFrame(earningsData['quarterlyEarnings'])
    incomeDf = pd.DataFrame(incomeData['quarterlyReports'])
    statsDf = createStatsDf(earningsDf, incomeDf, TICKER)
    
    predictDf = createFinalDf(statsDf, dailyStock)
    predictDf = predictDf.sort_values(by='Earnings_Date', ascending=False)
    returns, sampleSize = rangeSearch(predictDf.iloc[0]['Earn_Increase']-0.05, predictDf.iloc[0]['Earn_Increase']+0.05, predictDf.iloc[0]['Rev_Increase']-0.01, predictDf.iloc[0]['Rev_Increase']+0.01, predictDf.iloc[0]['Earn_Increase_Count'])
    result = {
        'returns': str(round(returns * 100, 2)),
        'earnIncrease': str(round(predictDf.iloc[0]['Earn_Increase']*100, 2)),
        'revIncrease': str(round(predictDf.iloc[0]['Rev_Increase']*100, 2)),
        'earnIncreaseCount': str(int(predictDf.iloc[0]['Earn_Increase_Count']))
    }
    return result
#     return returns

In [26]:
def getDailyStockPrices(TICKER):
    keyDict = {
        '1. open': 'Open',
        '2. high': 'High',
        '3. low': 'Low',
        '4. close': 'Close',
        '5. volume': 'Volume'
    }
    timeSeriesUrl = getUrl('TIME_SERIES_DAILY', TICKER)
    timeData = getDataFromAlphaVantage(timeSeriesUrl).json();
    timeData = timeData['Time Series (Daily)']
    listOfDicts = [{'Date': key, **value} for key, value in timeData.items()]
    listOfDicts = [
        {keyDict.get(k, k): v for k, v in d.items()} 
        for d in listOfDicts
    ]
    return listOfDicts[:30]
#     dailyStock = createDailyDf(timeData, TICKER)
#     return dailyStock
    

In [44]:
print(getDataFromAlphaVantage.cache_info())

CacheInfo(hits=5, misses=12, maxsize=None, currsize=12)


In [ ]:
app = Flask(__name__)
CORS(app)

@app.route('/api/hello', methods=['GET'])
def hello_world():
    return jsonify({"message": "Hello, World!"})

@app.route('/predict', methods=['GET'])
def predictReturns():
    ticker = request.args.get('ticker')
    return jsonify(predictStockReturns(ticker))

@app.route('/prices/daily', methods=['GET'])
def getPricesDaily():
    ticker = request.args.get('ticker')
    return jsonify(getDailyStockPrices(ticker))

if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [07/Sep/2024 11:34:47] "GET /predict?ticker=MSFT HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2024 11:34:47] "GET /prices/daily?ticker=MSFT HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2024 11:36:51] "GET /predict?ticker=NFLX HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2024 11:36:51] "GET /prices/daily?ticker=NFLX HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2024 11:40:35] "GET /predict?ticker=MSFT HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2024 11:40:35] "GET /prices/daily?ticker=MSFT HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2024 11:42:44] "GET /predict?ticker=MSFT HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2024 11:42:44] "GET /prices/daily?ticker=MSFT HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2024 11:43:50] "GET /predict?ticker=NFLX HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2024 11:43:50] "GET /prices/daily?ticker=NFLX HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2024 11:55:04] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [07/Sep/2024 11:55:26] "GET / HTTP/1.1" 404 -
127.0